In [48]:
import os
from nd2reader import ND2Reader
import napari
import numpy as np
from cellpose import models
from skimage.measure import regionprops_table
from skimage.io import imsave, imread
from skimage.segmentation import clear_border
from skimage.transform import resize
from skimage.morphology import remove_small_objects
import pandas as pd
from skimage.feature import blob_dog
from utils import sharpest_frame_laplacian

In [53]:
data_dir = r'I:\CBI\Kasia\data_analysis\2025_Fouquerel\data\53BP1 IF cenpB FISH -Lily'
masks_dir = r'I:\CBI\Kasia\data_analysis\2025_Fouquerel\analysis\250320_nuclei_masks'
file_name = 'Batch Deconvolution_000_clone14_untransfected_20minDL_24h_005.nd2'

In [56]:
df_foci1

,y,x,r,cell
0,1893.0,1239.0,2.0,13
1,1560.0,1417.0,2.0,8
2,1909.0,1646.0,2.0,11
3,1864.0,664.0,2.0,12
4,1894.0,578.0,2.0,12
...,...,...,...,...
322,330.0,1000.0,2.0,1
323,1562.0,1893.0,2.0,9
324,1238.0,462.0,2.0,7
325,531.0,1935.0,2.0,3


In [57]:
# read in the data

df_cell = pd.read_csv(os.path.join(masks_dir, file_name.replace('.nd2', '_cell_data.csv')))
df_foci1 = pd.read_csv(os.path.join(masks_dir, file_name.replace('.nd2', '_foci1_data.csv')))
df_foci2 = pd.read_csv(os.path.join(masks_dir, file_name.replace('.nd2', '_foci2_data.csv')))

# drop foci from the background
df_foci1 = df_foci1[df_foci1['cell'] != 0]
df_foci2 = df_foci2[df_foci2['cell'] != 0]

masks = imread(os.path.join(masks_dir, file_name.replace('.nd2', '_mask.png')))

im_path = os.path.join(data_dir, file_name)
t = ND2Reader(im_path)
sharpest_index = df_cell.loc[0,'frame']
dapi_im = t.get_frame_2D(c=0, z=sharpest_index)
foci1_im = t.get_frame_2D(c=1, z=sharpest_index)
foci2_im = t.get_frame_2D(c=2, z=sharpest_index)

dapi_stack = np.array([t.get_frame_2D(c=0, z=z) for z in range(t.sizes['z'])])
foci1_stack = np.array([t.get_frame_2D(c=1, z=z) for z in range(t.sizes['z'])])
foci2_stack = np.array([t.get_frame_2D(c=2, z=z) for z in range(t.sizes['z'])])

In [58]:
viewer = napari.Viewer()
viewer.add_image(dapi_stack,blending='additive', colormap='blue', name='DAPI',visible=False)
viewer.add_image(foci1_stack,blending='additive', colormap='green', name='foci1', visible = True)
viewer.add_image(foci2_stack,blending='additive', colormap='red', name='foci2',visible = False)

viewer.add_points(df_foci1.loc[:,['y','x']], size=5, face_color='green', name='foci1')
viewer.add_points(df_foci2.loc[:,['y','x']], size=5, face_color='red', name='foci2')

viewer.add_labels(masks, name='masks', visible=False)

# set slider position
viewer.dims.set_point(0, sharpest_index)

In [59]:
df_foci2_fake = pd.read_csv(os.path.join(masks_dir, file_name.replace('.nd2', '_foci2_ctrl_data.csv')))

# drop foci from the background
df_foci2_fake = df_foci2_fake[df_foci2_fake['cell'] != 0]

In [60]:
viewer.add_points(df_foci2_fake.loc[:,['y','x']], size=5, face_color='white', name='foci2_ctrl')

<Points layer 'foci2_ctrl' at 0x198228ebb10>